<a href="https://colab.research.google.com/github/brodyandre/regex_em_dados_textuais_json_de_vendas/blob/main/regex_em_dados_textuais_json_de_vendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📝 Documentação - Limpeza e Transformação de Dados de Vendas

Esta documentação descreve de forma estruturada e profissional o processo de limpeza e transformação de dados de vendas extraídos de um arquivo JSON.

###1️⃣ Instalação de Dependências

Para realizar o processamento dos dados, é necessário instalar a biblioteca pandas:

In [15]:
!pip install pandas

###2️⃣ Importação de Bibliotecas

As bibliotecas necessárias para manipulação de dados e tratamento de strings são importadas:

In [16]:
import numpy as np
import json
import pandas as pd
import re
import unicodedata

###3️⃣ Leitura do Arquivo JSON

O arquivo JSON é carregado e convertido para um DataFrame do Pandas:

In [17]:
with open('/content/dados_vendas_clientes.json', 'r') as file:
    dados = json.load(file)

dados = pd.DataFrame(dados)
dados.head()

,dados_vendas
0,"{'Data de venda': '06/06/2022', 'Cliente': ['@..."
1,"{'Data de venda': '07/06/2022', 'Cliente': ['I..."
2,"{'Data de venda': '08/06/2022', 'Cliente': ['I..."
3,"{'Data de venda': '09/06/2022', 'Cliente': ['J..."
4,"{'Data de venda': '10/06/2022', 'Cliente': ['M..."


###4️⃣ Normalização dos Dados

Para acessar corretamente as informações, é realizada a normalização dos dados:

In [18]:
dados = pd.json_normalize(dados['dados_vendas'])
dados

,Data de venda,Cliente,Valor da compra
0,06/06/2022,"[@ANA _LUCIA 321, DieGO ARMANDIU 210, DieGO AR...","[R$ 836,5, R$ 573,33, R$ 392,8, R$ 512,34]"
1,07/06/2022,"[Isabely JOanes 738, Isabely JOanes 738, Isabe...","[R$ 825,31, R$ 168,07, R$ 339,18, R$ 314,69]"
2,08/06/2022,"[Isabely JOanes 738, JOãO Gabriel 671, Julya m...","[R$ 682,05, R$ 386,34, R$ 622,65, R$ 630,79]"
3,09/06/2022,"[Julya meireles 914, MaRIA Julia 444, MaRIA Ju...","[R$ 390,3, R$ 759,16, R$ 334,47, R$ 678,78]"
4,10/06/2022,"[MaRIA Julia 444, PEDRO PASCO 812, Paulo castr...","[R$ 314,24, R$ 311,15, R$ 899,16, R$ 885,24]"


###5️⃣ Renomeação das Colunas

As colunas são renomeadas para nomes mais intuitivos:

In [19]:
colunas = list(dados.columns)
dados = dados.rename(columns={
    colunas[0]: 'data_da_venda',
    colunas[1]: 'cliente',
    colunas[2]: 'valor_da_compra'
})
dados

,data_da_venda,cliente,valor_da_compra
0,06/06/2022,"[@ANA _LUCIA 321, DieGO ARMANDIU 210, DieGO AR...","[R$ 836,5, R$ 573,33, R$ 392,8, R$ 512,34]"
1,07/06/2022,"[Isabely JOanes 738, Isabely JOanes 738, Isabe...","[R$ 825,31, R$ 168,07, R$ 339,18, R$ 314,69]"
2,08/06/2022,"[Isabely JOanes 738, JOãO Gabriel 671, Julya m...","[R$ 682,05, R$ 386,34, R$ 622,65, R$ 630,79]"
3,09/06/2022,"[Julya meireles 914, MaRIA Julia 444, MaRIA Ju...","[R$ 390,3, R$ 759,16, R$ 334,47, R$ 678,78]"
4,10/06/2022,"[MaRIA Julia 444, PEDRO PASCO 812, Paulo castr...","[R$ 314,24, R$ 311,15, R$ 899,16, R$ 885,24]"


###6️⃣ Tratamento de Dados

Para manipular listas aninhadas, o método explode() é utilizado, seguido do reset do índice:

In [20]:
colunas = list(dados.columns)
dados = dados.explode(colunas[1:])
dados.reset_index(drop=True, inplace=True)
dados

,data_da_venda,cliente,valor_da_compra
0,06/06/2022,@ANA _LUCIA 321,"R$ 836,5"
1,06/06/2022,DieGO ARMANDIU 210,"R$ 573,33"
2,06/06/2022,DieGO ARMANDIU 210,"R$ 392,8"
3,06/06/2022,DieGO ARMANDIU 210,"R$ 512,34"
4,07/06/2022,Isabely JOanes 738,"R$ 825,31"
5,07/06/2022,Isabely JOanes 738,"R$ 168,07"
6,07/06/2022,Isabely JOanes 738,"R$ 339,18"
7,07/06/2022,Isabely JOanes 738,"R$ 314,69"
8,08/06/2022,Isabely JOanes 738,"R$ 682,05"
9,08/06/2022,JOãO Gabriel 671,"R$ 386,34"


###7️⃣ Remoção de Acentos e Caracteres Especiais

É implementada uma função para remover acentuação e caracteres especiais dos nomes dos clientes, convertendo todos os nomes para  letras minúsculas:

In [21]:
def remove_acentos_caracteres_especiais(texto):
    # Remover acentos
    nfkd = unicodedata.normalize('NFKD', texto)
    texto_sem_acentos = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    # Remover caracteres especiais e números, e converter para minúsculas
    texto_limpo = re.sub(r'[^a-zA-Z\s]', '', texto_sem_acentos).lower()
    return texto_limpo

dados['cliente'] = dados['cliente'].apply(remove_acentos_caracteres_especiais)
dados

,data_da_venda,cliente,valor_da_compra
0,06/06/2022,ana lucia,"R$ 836,5"
1,06/06/2022,diego armandiu,"R$ 573,33"
2,06/06/2022,diego armandiu,"R$ 392,8"
3,06/06/2022,diego armandiu,"R$ 512,34"
4,07/06/2022,isabely joanes,"R$ 825,31"
5,07/06/2022,isabely joanes,"R$ 168,07"
6,07/06/2022,isabely joanes,"R$ 339,18"
7,07/06/2022,isabely joanes,"R$ 314,69"
8,08/06/2022,isabely joanes,"R$ 682,05"
9,08/06/2022,joao gabriel,"R$ 386,34"


###8️⃣ Formatação dos Valores Monetários

Os valores de compra são formatados para tipo float, removendo o símbolo "R$" e ajustando a pontuação:

In [22]:
dados['valor_da_compra'] = dados['valor_da_compra'].str.replace('R$', '', regex=False)
dados['valor_da_compra'] = dados['valor_da_compra'].str.replace('.', '', regex=False)
dados['valor_da_compra'] = dados['valor_da_compra'].str.replace(',', '.', regex=False)
dados['valor_da_compra'] = dados['valor_da_compra'].astype(float)
dados

,data_da_venda,cliente,valor_da_compra
0,06/06/2022,ana lucia,836.50
1,06/06/2022,diego armandiu,573.33
2,06/06/2022,diego armandiu,392.80
3,06/06/2022,diego armandiu,512.34
4,07/06/2022,isabely joanes,825.31
5,07/06/2022,isabely joanes,168.07
6,07/06/2022,isabely joanes,339.18
7,07/06/2022,isabely joanes,314.69
8,08/06/2022,isabely joanes,682.05
9,08/06/2022,joao gabriel,386.34


###9️⃣ Convertendo a Coluna data_da_venda para o tipo datetime

A estrutura final dos dados é exibida, pronta para análise:

In [23]:
dados['data_da_venda'] = pd.to_datetime(dados['data_da_venda'])
dados.info()
dados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   data_da_venda    20 non-null     datetime64[ns]
 1   cliente          20 non-null     object        
 2   valor_da_compra  20 non-null     float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 612.0+ bytes


,data_da_venda,cliente,valor_da_compra
0,2022-06-06,ana lucia,836.50
1,2022-06-06,diego armandiu,573.33
2,2022-06-06,diego armandiu,392.80
3,2022-06-06,diego armandiu,512.34
4,2022-07-06,isabely joanes,825.31
5,2022-07-06,isabely joanes,168.07
6,2022-07-06,isabely joanes,339.18
7,2022-07-06,isabely joanes,314.69
8,2022-08-06,isabely joanes,682.05
9,2022-08-06,joao gabriel,386.34


###🎯 Conclusão:
Os dados estão organizados, limpos e prontos para serem analisados, garantindo maior precisão nas operações futuras.